In [2]:
# Create folder to make inside them the videos
!mkdir REFERENCE_VIDEOS
!mkdir DISTORTED_VIDEOS

In [ ]:
# LPIPS For the objective score calculation
!pip install lpips

In [ ]:
import os
import tensorflow as tf
import cv2
import pandas as pd
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import subprocess
from tqdm import tqdm
from lpips import lpips

# AlexNet => Best Forward, see the documentation
loss_fn = lpips.LPIPS(net='alex')

In [ ]:
def read_frame_extract_patches(ref_frame, dis_frame):

  # /ref will contain the refernces frames (it will be consistent for all the distorted videos of the refernce video)
  # read refernce frame
  ref_image = Image.open('ref/' + ref_frame)
  
  # read distorted frame
  dis_image = Image.open(dis_frame).convert('RGB')
  
  # if the frame shape is not [1920*1080] then resize it using BUCIBIC Filter
  height = dis_frame.split('_')[4].split('x')[1]
  if (height != 1080):
    dis_image = dis_image.resize((1920, 1080), resample=Image.BICUBIC)
  
  # convert to numpy array
  dis_img = np.array(dis_image)
  ref_img = np.array(ref_image)

  # convert from 1920*1090*3 to 1*1920*1080*3
  ref_img = tf.expand_dims(ref_img, axis=0)
  dis_img = tf.expand_dims(dis_img, axis=0)

  #cast type to float32
  ref_img = tf.cast(ref_img, dtype=tf.float32)
  dis_img = tf.cast(dis_img, dtype=tf.float32)
  

  # Extract the patches from the images
  patch_size = (299, 299, 3)
  # no overlapping patches
  strides = (1, 299, 299, 1)
  dis_patches = tf.image.extract_patches(dis_img, sizes=[1, 299, 299, 1], strides=strides, rates=[1, 1, 1, 1], padding='VALID')
  ref_patches = tf.image.extract_patches(ref_img, sizes=[1, 299, 299, 1], strides=strides, rates=[1, 1, 1, 1], padding='VALID')

  # Reshape the patches to 18x299x299x3
  dis_patches = tf.reshape(dis_patches, [-1, *patch_size])
  ref_patches = tf.reshape(ref_patches, [-1, *patch_size])

  return ref_patches, dis_patches # need to be divided by 255.

In [ ]:
def calculate_lpips(references, distorted):
  
  # refernces patches
  references = references.numpy()/255.
  # distorted patches
  distorted = distorted.numpy()/255.

  #normalize to [-1 1], see documentation
  references = references  * 2.0 - 1.0
  distorted = distorted  * 2.0 - 1.0

  #convert tp pytroch tensor
  references = torch.tensor(references.transpose(0, 3, 1, 2))
  distorted = torch.tensor(distorted.transpose(0, 3, 1, 2))

  #lpips forward
  distance = loss_fn.forward(references, distorted)

  return distance[:, 0, 0, 0]

In [ ]:
# Read the videos files names
# References
ref_videos = os.listdir('REFERNCES_VIDEOS')

# Disrtorted by subset dataframe or by folder
dis_videos = pd.read_csv('selected_ditorted.csv')['Video'].values
# dis_videos = os.listdir('DISTIRTED_VIDEOS')


In [ ]:
# 'number' is only to say 'Patches_1' or 'Patches_2'...
def one_frame_processing(reference_frame, distorted_frame, number):


  output_patches_folder = f'Patches_{number}/'

  ref_patches, dis_patches = read_frame_extract_patches(reference_frame, distorted_frame)
  
  patches_lpips_scores = calculate_lpips(ref_patches, dis_patches)

  # '/255', to save them
  dis_patches = dis_patches / 255.

  patches_names = []
  for j, (patch) in enumerate(dis_patches):

    # To delete the '.png' extension
    distorted_frame_name = distorted_frame.split('.')[0]

    patch_name = distorted_frame_name + f'_patch_{j+1}.png'
    patch_path = output_patches_folder + patch_name
    plt.imsave(patch_path, patch.numpy(), format='png')

    patches_names.append(patch_name)

  return patches_names, patches_lpips_scores

In [ ]:
def Videos_process(reference_videos, distorted_videos, number):
  

  
  patches_csv = f'patches_{number}.csv'
  ref_path = 'REFERENCE_VIDEOS'
  dis_path = 'DISTORTED_VIDEOS'
  
  tqdm_bar = tqdm(reference_videos)
  header_patch = ['Patch', 'Lpips Score']

  with open(patches_csv, 'a') as w:
    w.write(','.join(header_patch) + '\n')
    for ref_video in tqdm_bar:
      ref_video_name_content = ref_video.split('_')

      # Each video is for a specific game name
      game_name = ref_video_name_content[0]
      # In one dataset, there is two part fro each game, if there is only one part this line can be deleted
      game_part = ref_video_name_content[3].split('.')[0]

      # To get all the distirted videos belong to the refernce video
      distorted = []
      for dis_video in distorted_videos:
        
        if (dis_video.split('_')[0] == game_name and dis_video.split('_')[3] == game_part):
          distorted.append(dis_video)
        
        # In the case of a video game has one part only
        # if (dis_video.split('_')[0] == game_name):
        #   distorted.append(dis_video)
          
      # ref is true means a new reference video with new disrtorted videos.  
      ref = True

      # the ref folder will contain all the refernce video frames for all the disrtoed videos
      os.system('mkdir ref')
      for i, dis_video in enumerate(distorted):
        cap = cv2.VideoCapture(os.path.join(dis_path, dis_video))
        video_frames = 900 # 30 second * 30 frame per second
        
        # the step is 12 , so take every 13th frame
        for frame in range(0, video_frames, 12):

          
          if(ref):
            # extract refernce frame and save it
            ref_frame = ref_video.split('.')[0] + f'_{frame}.png'
            extract_ref_png_frame = ['ffmpeg', '-y', '-s', f'{1920}x{1080}', '-pix_fmt', 'yuv420p', '-i', os.path.join(ref_path, ref_video), '-vf', f'select=eq(n\,{frame})', '-vframes', '1', ref_frame]
            subprocess.call(extract_ref_png_frame)
            os.system(f'mv {ref_frame} /content/ref/{ref_frame}')
              
          # for the distorted frame name 
          dis_frame = dis_video.split('.')[0] + f'_{frame}.png'        

          
          # save the disrtoted frame
          cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
          ret, img = cap.read()
          plt.imsave(dis_frame, img) 
          dis_image = Image.open(dis_frame)
          height = dis_frame.split('_')[4].split('x')[1]
          if (height != 1080):
            dis_image = dis_image.resize((1920, 1080), resample=Image.BICUBIC)
            
          patches_names, lpips_scores = one_frame_processing(ref_video.split('.')[0] + f'_{frame}.png', dis_frame, number)

          # save the patches name with their scores to the csv file
          for (patch, score) in zip(patches_names, lpips_scores):
            data = ','.join([patch, str(score.item())]) + '\n'
            w.write(data)

          # delete the processed distorted frame
          os.system(f'rm {dis_frame}')

          # update the result to the plot bar
          tqdm_bar.set_postfix({'distorted_video_processed': f'{i}/{len(distorted)}', 'frame processed': frame})
        # setting ref = false , will stop the processing of the ref video as it is ready for the next distorted video
        ref = False
        
        # close the video
        cap.release()
      # delete the ref video
      os.system('rm -r ref')

In [ ]:
number = 1
os.system(f'mkdir Patches_{1}')
Videos_process(ref_videos, dis_videos, number)

100%|██████████| 4/4 [40:24<00:00, 606.22s/it, dis=2/3, frame=888]


In [ ]:
# Zip the result folder
os.system(f'zip -r Patches_{number}.zip Patches_{number}')